In [1]:
import datetime
import sqlite3
import pandas as pd
import re

pd.set_option("display.max_rows", None)

try:
    connection = sqlite3.connect("../local/local.sqlite3")
    df = pd.read_sql_query("select * from property", connection)
finally:
    connection.close()


def get_suburb(address: str):
    address_split = address.split(",") if address else []
    return (
        address_split[len(address_split) - 1].strip()
        if len(address_split) > 0
        else address
    )


df["suburb"] = df["address"].apply(get_suburb)
suburb_col = df.pop("suburb")
df.insert(10, "suburb", suburb_col)


def extract_price(price_detail: str) -> int | None:
    def price_str_to_num(price_str):
        price_str = price_str.replace(" ", "")
        price_str = price_str.lower().replace(",", "")
        if "k" in price_str:
            price_str = price_str.replace("k", "")
            return int(price_str) * 1000
        if "K" in price_str:
            price_str = price_str.replace("K", "")
            return int(price_str) * 1000
        if "s" in price_str:
            price_str = price_str.replace("s", "")
            if len(price_str) <= 3:
                return int(price_str) * 1000
            return int(price_str)
        return int(price_str) if price_str and price_str.strip() != "" else None

    patterns = [
        r".?\$?([0-9,]+[Kk]?)\s*-\s*\$?([0-9,]+[Kk]?)",
        r"\$([0-9, ]+[Kks]?)",
        r"^([0-9, ]+[Kks]?)",
    ]
    for pattern in patterns:
        match = re.search(pattern, price_detail, re.IGNORECASE)
        if match:
            if len(match.groups()) == 2:
                price1 = price_str_to_num(match.group(1))
                price2 = price_str_to_num(match.group(2))
                return int((price1 + price2) / 2)
            else:
                return price_str_to_num(match.group(1))


df["price"] = df["price_detail"].apply(extract_price)
price_col = df.pop("price")
df.insert(9, "price", price_col)

df.to_excel(f"../local/local-{datetime.datetime.now().date()}.xlsx", index=False)


def get_median_prices_with_counts(
    rent_property_df: pd.DataFrame, buy_property_df: pd.DataFrame
) -> pd.DataFrame:
    median_rent_property_prices = (
        rent_property_df.groupby(["query_city", "suburb"])["price"]
        .median()
        .reset_index()
    )
    median_rent_property_prices = median_rent_property_prices.rename(
        columns={"price": "median_price"}
    )
    median_buy_property_prices = (
        buy_property_df.groupby(["query_city", "suburb"])["price"]
        .median()
        .reset_index()
    )
    median_buy_property_prices = median_buy_property_prices.rename(
        columns={"price": "median_price"}
    )
    median_property_prices = median_rent_property_prices.merge(
        median_buy_property_prices,
        on=["query_city", "suburb"],
        suffixes=("_rent", "_buy"),
    )
    count_rent_properties = (
        rent_property_df.groupby(["query_city", "suburb"])["url"].count().reset_index()
    )
    count_rent_properties = count_rent_properties.rename(columns={"url": "count"})
    count_buy_properties = (
        buy_property_df.groupby(["query_city", "suburb"])["url"].count().reset_index()
    )
    count_buy_properties = count_buy_properties.rename(columns={"url": "count"})
    count_properties = count_rent_properties.merge(
        count_buy_properties, on=["query_city", "suburb"], suffixes=("_rent", "_buy")
    )
    properties = count_properties.merge(
        median_property_prices, on=["query_city", "suburb"]
    )
    properties["gross_rental_yield"] = round(
        ((properties["median_price_rent"] * 52) / properties["median_price_buy"]) * 100,
        2,
    )
    return properties


rent_house_df = df[(df["listing_type"] == "RENT") & (df["property_type"] == "HOUSE")]
buy_house_df = df[(df["listing_type"] == "BUY") & (df["property_type"] == "HOUSE")]
houses_df = get_median_prices_with_counts(rent_house_df, buy_house_df)
houses_df.to_excel(
    f"../local/houses-{datetime.datetime.now().date()}.xlsx", index=False
)

rent_townhouse_df = df[
    (df["listing_type"] == "RENT") & (df["property_type"] == "TOWNHOUSE")
]
buy_townhouse_df = df[
    (df["listing_type"] == "BUY") & (df["property_type"] == "TOWNHOUSE")
]
townhouses_df = get_median_prices_with_counts(rent_townhouse_df, buy_townhouse_df)
townhouses_df.to_excel(
    f"../local/townhouses-{datetime.datetime.now().date()}.xlsx", index=False
)


# import matplotlib.pyplot as plt
# import seaborn as sns

# rent_df = df[(df["listing_type"] == "RENT")]
# buy_df = df[(df["listing_type"] == "BUY")]

# avg_rent_prices = rent_df.groupby("suburb")["price"].mean().reset_index()
# avg_buy_prices = buy_df.groupby("suburb")["price"].mean().reset_index()

# avg_prices = avg_rent_prices.merge(
#     avg_buy_prices, on="suburb", suffixes=("_rent", "_buy")
# )

# median_prices_melted = median_prices.melt(
#     id_vars="suburb",
#     value_vars=["price_rent", "price_buy"],
#     var_name="listing_type",
#     value_name="median_price",
# )

# plt.figure(figsize=(12, 6))
# sns.barplot(
#     data=median_prices,
#     x="suburb",
#     y="median_price",
#     hue="listing_type",
#     palette=["red", "green"],
# )
# plt.title("Median Property Prices by Suburb and Listing Type")
# plt.xlabel("Suburb")
# plt.ylabel("Median Price")
# plt.legend(title="Listing Type", labels=["Rent", "Buy"])

# sns.histplot(rent_df["price"], kde=True, color="blue", label="Rental prices")
# sns.histplot(buy_df["price"], kde=True, color="green", label="Buy prices")
# plt.title("Distribution of Property Prices")
# plt.xlabel("Price")
# plt.ylabel("Frequency")
# plt.legend()

# plt.show()